
# 📘 Marco Teórico de Métricas Financieras

---

## 1️⃣ Alpha (α)
* **Definición:** mide el exceso de retorno respecto al esperado por el CAPM. (Viene adicional en la última parte)
* **Fórmula:**
  $$
  \alpha = R_p - \left[R_f + \beta_p (R_m - R_f)\right]
  $$
* **Interpretación:**
  * $\alpha > 0$: valor agregado (rendimiento superior al esperado).
  * $\alpha < 0$: rendimiento inferior al esperado.
* **Óptimo:** +2% a +10% anual.

---

## 2️⃣ Beta (β)
* **Definición:** sensibilidad del portafolio frente al mercado.
* **Fórmula:**
  $$
  \beta_p = \frac{Cov(R_p, R_m)}{Var(R_m)}
  $$

Define:

*	$Cov(R_p, R_m)$ → es la covarianza entre los rendimientos del portafolio ($R_p$) y los del mercado ($R_m$).
Mide cómo se mueven en conjunto.
* $Var(R_m)$ → es la varianza del mercado (benchmark).
Mide la dispersión de los rendimientos del mercado respecto a su media.

* **Interpretación:**
  * $\beta \approx 1$: portafolio se mueve igual que el mercado.
  * $\beta > 1$: más volátil (agresivo).
  * $\beta < 1$: más defensivo y estable.

---

## 3️⃣ R² (Coeficiente de Determinación)
* **Definición:** mide cuánto del movimiento del portafolio se explica por el benchmark.
* **Fórmula:**
  $$
  R^2 = \left( \frac{Cov(R_p, R_m)}{\sigma_p \cdot \sigma_m} \right)^2
  $$
* **Interpretación:**
  * $R^2 > 0.8$: portafolio muy indexado.
  * $0.6 \leq R^2 \leq 0.8$: buena relación con el mercado.
  * $R^2 < 0.3$: portafolio muy independiente.

---

## 4️⃣ Retorno Anualizado (µ)
* **Definición:** rendimiento promedio anual esperado del portafolio.
* **Fórmula:**
  $$
  \mu_{anual} = (1 + \mu_{diario})^{252} - 1
  $$
* **Interpretación:**
  * $> 8\%$: atractivo y estable.
  * $> 15\%$: agresivo (con mayor riesgo).

---

## 5️⃣ Volatilidad Anualizada (σ)
* **Definición:** mide el riesgo total (desviación estándar de retornos).
* **Fórmula:**
  $$
  \sigma_{anual} = \sigma_{diaria} \cdot \sqrt{252}
  $$
* **Interpretación:**
  * $\sigma < 10\%$: defensivo.
  * $10\% \leq \sigma \leq 20\%$: moderado.
  * $\sigma > 25\%$: muy riesgoso.

---

## 6️⃣ Sharpe Ratio
* **Definición:** retorno ajustado por riesgo total.
* **Fórmula:**
  $$
  Sharpe = \frac{R_p - R_f}{\sigma_p}
  $$
* **Interpretación:**
  * $Sharpe > 1$: bueno.
  * $Sharpe > 1.5$: muy bueno.
  * $Sharpe > 2$: excelente.

---

## 7️⃣ Sortino Ratio
* **Definición:** retorno ajustado solo por el riesgo a la baja.
* **Fórmula:**
  $$
  Sortino = \frac{R_p - R_f}{\sigma_{downside}\text{ (or DD)}}
  $$
* **Interpretación:**
  * $Sortino > 2$: excelente (mejor que Sharpe si las caídas son limitadas).
  * A higher Sortino ratio is better, indicating that an investment is earning a greater return for each unit of negative risk it takes on.
  * The ratio is particularly useful for risk-averse investors, as it provides a more accurate picture of risk by separating potentially harmful volatility from upside volatility
  * DD - downside deviation, which is the standard deviation of only the negative returns.

---

## 8️⃣ Value at Risk (VaR)
* **Definición:** pérdida máxima esperada con cierto nivel de confianza.
* **Fórmula (95%, 1 día):**
  $$
  VaR_{95\%,1d} = \mu - Z_{0.95} \cdot \sigma
  $$
* **Interpretación:**
  * “Con 95% de confianza, no se perderá más de X% en un día”.
  * Óptimo: $VaR < 2\%$.

---

## 9️⃣ Conditional Value at Risk (CVaR)
* **Definición:** pérdida promedio esperada en los peores escenarios (cola izquierda).
* **Fórmula:**
  $$
  CVaR = E[\text{pérdida} \mid \text{pérdida} > VaR]
  $$
* **Interpretación:**
  * Visión más conservadora que el VaR.
  * Óptimo: $CVaR < 3\%$ (95%, 1 día).

---


In [18]:
# @title
# !pip install yfinance scipy ipywidgets --quiet

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from scipy.stats import norm

import ipywidgets as w
from IPython.display import display, clear_output

# ===================== Utilidades =====================

def download_close(tickers, start, end, auto_adjust=True):
    df = yf.download(tickers, start=start, end=end, auto_adjust=auto_adjust, progress=False)
    if isinstance(df.columns, pd.MultiIndex):
        close = df['Close'].copy()
    else:
        # Caso de un único ticker -> normalizamos a DataFrame con una columna
        close = df[['Close']].copy()
        colname = tickers if isinstance(tickers, str) else tickers[0]
        close.columns = [colname]
    close = close.dropna(how='all').dropna(axis=1, how='all')
    return close

def download_bench(bench, start, end, auto_adjust=True):
    s = yf.download(bench, start=start, end=end, auto_adjust=auto_adjust, progress=False)
    if isinstance(s.columns, pd.MultiIndex):
      s.columns = s.columns.get_level_values(0)
    s = s['Close'].copy()
    s = pd.Series(s, name=str(bench)).dropna()
    return s

def compute_returns(prices: pd.DataFrame | pd.Series, kind="log"):
    if isinstance(prices, pd.Series):
        if kind=="log":
            return np.log(prices/prices.shift(1)).dropna()
        return prices.pct_change().dropna()
    if kind=="log":
        return np.log(prices/prices.shift(1)).dropna()
    return prices.pct_change().dropna()

def annualize_stats(rets: pd.DataFrame, periods=252):
    mu = rets.mean()*periods
    cov = rets.cov()*periods
    return mu, cov

def optimize_sharpe(rets: pd.DataFrame, rf=0.0, periods=252, n_starts=25, long_only=True, seed=123):
    rng = np.random.default_rng(seed)
    mu_ann, cov_ann = annualize_stats(rets, periods)
    n = rets.shape[1]
    bounds = [(0,1)]*n if long_only else [(-1,1)]*n
    cons = ({'type':'eq','fun':lambda w: np.sum(w)-1},)
    inits = [np.ones(n)/n]
    if long_only:
        inits += [rng.dirichlet(np.ones(n)) for _ in range(max(0, n_starts-1))]
    else:
        for _ in range(n_starts-1):
            w0 = rng.normal(0,0.5,n)
            w0 = w0/np.sum(np.abs(w0))
            inits.append(w0)
    def neg_sharpe(w):
        mu = float(w@mu_ann.values)
        vol = float(np.sqrt(w@cov_ann.values@w))
        return -((mu-rf)/vol if vol>0 else -1e9)
    best=None; bestf=np.inf
    for x0 in inits:
        res=minimize(neg_sharpe, x0, method='SLSQP', bounds=bounds, constraints=cons,
                     options={'maxiter':500,'ftol':1e-12})
        if res.success and res.fun<bestf:
            best, bestf = res, res.fun
    if best is None: raise RuntimeError("No se encontró solución factible.")
    w_opt = pd.Series(best.x, index=rets.columns)
    mu_p = float(w_opt.values @ mu_ann.values)
    vol_p = float(np.sqrt(w_opt.values @ cov_ann.values @ w_opt.values))
    sr = (mu_p - rf)/vol_p if vol_p>0 else np.nan
    return {"weights": w_opt, "mu_ann": mu_p, "vol_ann": vol_p, "sharpe": sr,
            "mu_vec": mu_ann, "cov": cov_ann}

def jensen_alpha_beta(port_ret, bench_ret, rf_annual=0.0, periods=252):
    """Alpha (Jensen) y Beta (CAPM) por regresión de excesos diarios, alpha anualizada."""
    rf_d = rf_annual/periods
    x = (bench_ret - rf_d).dropna()
    y = (port_ret  - rf_d).dropna()
    df = pd.concat([x,y], axis=1).dropna()
    if df.empty: return np.nan, np.nan, np.nan
    xb = df.iloc[:,0].values
    yb = df.iloc[:,1].values
    # Regresión OLS con intercepto: y = a + b x
    b, a = np.polyfit(xb, yb, 1)
    # alpha diario = a; beta = b
    alpha_ann = a * periods
    # R^2
    r = np.corrcoef(xb, yb)[0,1] if len(df)>1 else np.nan #Pearson Correlation coefficient
    r2 = r**2 if pd.notna(r) else np.nan
    return alpha_ann, b, r2

def sharpe_ratio_series(ret, rf_annual=0.0, periods=252):
    mu_ann = ret.mean()*periods
    vol_ann = ret.std(ddof=1)*np.sqrt(periods)
    rf = rf_annual
    return (mu_ann - rf)/vol_ann if vol_ann>0 else np.nan

def sortino_ratio_series(ret, rf_annual=0.0, periods=252):
    rf_d = rf_annual/periods
    ex = ret - rf_d #excess returns
    downside = np.minimum(ex, 0.0) #Isolate downside deviations 
    dd = downside.std(ddof=1) * np.sqrt(periods) #downside deviation (annualized)
    mu_ex_ann = ex.mean()*periods # Calculate annualized excess mean return
    return (mu_ex_ann)/dd if dd>0 else np.nan

def var_cvar(ret, level=0.95, method="historical", rf_annual=0.0, periods=252):
    """VaR/CVaR 1-día (no anualiza). Ret es serie diaria."""
    r = ret.dropna().values
    if r.size==0: return np.nan, np.nan
    alpha = level
    if method=="historical":
        q = np.quantile(r, 1-alpha)
        var_d = -q
        cvar_d = -r[r<=q].mean() if (r<=q).any() else np.nan
        return var_d, cvar_d
    # paramétrico (normal)
    mu_d, sd_d = np.mean(r), np.std(r, ddof=1)
    z = norm.ppf(1-alpha)
    var_d = -(mu_d + sd_d*z)
    cvar_d = -(mu_d - sd_d * (norm.pdf(z)/(1-alpha)))
    return var_d, cvar_d

def cum_return(ret):
    return (1+ret).cumprod()


In [19]:
# @title

# ===================== UI =====================

OPCIONES = ["AAPL","MSFT","GOOG","NVDA","META","AMZN","TSLA","BRK-B","V","JPM",
            "HWM","IBM","SPY",
            "XOM","LLY","UNH","PG","MA","KO","PEP","COST","ORCL","CSCO","NKE","ASML","TXN","ABT" ,"BTC-USD" , "ETH-USD", "USDT-USD", "XRP-USD", "LTC-USD", "ADA-USD", "DOT-USD", "BCH-USD", "XLM-USD", "LINK-USD"]

BENCH_OPC = ["^GSPC (S&P 500)","^IXIC (Nasdaq)","^DJI (Dow)","SPY","QQQ","EFA","EEM"]

tickers_w = w.SelectMultiple(options=OPCIONES, value=("AAPL","MSFT","GOOG"),
                             description="Tickers", rows=8, layout=w.Layout(width="300px"))
tickers_extra = w.Text(value="", description="Extra (coma)", placeholder="Ej: IBM, INTC, TSM")
bench_dd = w.Dropdown(options=BENCH_OPC, value="^GSPC (S&P 500)", description="Benchmark")
bench_txt = w.Text(value="", description="Custom", placeholder="Ej: ^MXX, VT")

start_w = w.DatePicker(description="Inicio", value=pd.Timestamp("2023-01-01").date())
end_w   = w.DatePicker(description="Fin",    value=pd.Timestamp("2025-08-22").date())
rf_w    = w.FloatText(description="rf anual", value=0.00, step=0.001)

retkind_dd = w.Dropdown(options=[("Log","log"),("Simple","simple")], value="log", description="Rend.")
weights_dd = w.Dropdown(options=[("Igual","equal"),("Optimizar Sharpe","opt")], value="opt", description="Pesos")

var_level_dd = w.Dropdown(options=[("90%",0.90),("95%",0.95),("99%",0.99)], value=0.95, description="VaR nivel")
var_method_dd= w.Dropdown(options=[("Histórico","historical"),("Paramétrico","parametric")],
                          value="historical", description="Método")

nstarts_w = w.IntSlider(description="Reinicios opt.", min=5, max=100, step=5, value=25)
seed_w    = w.IntText(description="Seed", value=123)

run_btn = w.Button(description="Calcular", button_style="primary", icon="calculator")
out = w.Output()

def _norm_t(t):
    return t.strip().upper().replace("BRK.B","BRK-B")

def run(_):
    with out:
        clear_output()
        # Tickers
        base = list(tickers_w.value)
        extra = [_norm_t(x) for x in tickers_extra.value.split(",")] if tickers_extra.value else []
        extra = [x for x in extra if x]
        tickers = list(dict.fromkeys([_norm_t(x) for x in base + extra]))
        if len(tickers) < 3:
            print("⚠️ Selecciona/ingresa al menos 3 tickers."); return

        # Benchmark
        bench_val = bench_dd.value.split()[0] if bench_dd.value else "^GSPC"
        if bench_txt.value.strip():
            bench_val = bench_txt.value.strip()

        start = str(start_w.value); end = str(end_w.value)
        rf = float(rf_w.value)
        kind = retkind_dd.value
        wmode = weights_dd.value
        var_level = float(var_level_dd.value)
        var_method = var_method_dd.value
        nstarts = int(nstarts_w.value)
        seed = int(seed_w.value)

        print("Descargando datos…")
        prices = download_close(tickers, start, end, auto_adjust=True)
        bench_close = download_bench(bench_val, start, end, auto_adjust=True)

        if prices.empty or bench_close.empty:
            print("⚠️ Datos insuficientes (activos o benchmark)."); return

        # Rendimientos
        rets = compute_returns(prices, kind)
        bench_ret = compute_returns(bench_close, kind)

        # Alinear fechas
        idx = rets.index.intersection(bench_ret.index)
        rets = rets.loc[idx]
        bench_ret = bench_ret.loc[idx]

        # Pesos
        if wmode == "equal":
            w_ = pd.Series(np.ones(rets.shape[1])/rets.shape[1], index=rets.columns)
            mu_ann, cov_ann = annualize_stats(rets)
            mu_p = float(w_.values @ mu_ann.values)
            vol_p = float(np.sqrt(w_.values @ cov_ann.values @ w_.values))
            sharpe_p = (mu_p - rf)/vol_p if vol_p>0 else np.nan
        else:
            res = optimize_sharpe(rets, rf=rf, periods=252, n_starts=nstarts, long_only=True, seed=seed)
            w_ = res["weights"]
            mu_p, vol_p, sharpe_p = res["mu_ann"], res["vol_ann"], res["sharpe"]

        # Retorno del portafolio diario
        port_ret = (rets @ w_.values)
        # Métricas principales
        sharpe_val = sharpe_ratio_series(port_ret, rf_annual=rf, periods=252)
        sortino_val = sortino_ratio_series(port_ret, rf_annual=rf, periods=252)
        alpha_ann, beta, r2 = jensen_alpha_beta(port_ret, bench_ret, rf_annual=rf, periods=252)

        # VaR / CVaR 1-día
        if var_method == "historical":
            var_d, cvar_d = var_cvar(port_ret, level=var_level, method="historical")
        else:
            var_d, cvar_d = var_cvar(port_ret, level=var_level, method="parametric")

        # Tabla de resultados
        mu_b = bench_ret.mean()*252
        vol_b = bench_ret.std(ddof=1)*np.sqrt(252)
        sharpe_b = (mu_b - rf)/(vol_b if vol_b>0 else np.nan)

        summary = pd.DataFrame({
            "μ anual": [mu_p, mu_b],
            "σ anual": [vol_p, vol_b],
            "Sharpe": [sharpe_val, sharpe_b],
            "Sortino": [sortino_val, np.nan],
        }, index=["Portafolio","Benchmark"]).round(4)

        capm = pd.DataFrame({
            "Alpha (anual)": [alpha_ann],
            "Beta": [beta],
            "R²": [r2],
            "VaR 1d": [var_d],
            "CVaR 1d": [cvar_d],
        }, index=["Portafolio"]).round(4)

        print("\nPesos del Portafolio (%)")
        print((w_*100).round(2).to_string())

        print("\nMétricas (anualizadas)")
        display(summary)
        print(f"Exceso rf usado: {rf:.2%} anual | Rend: {kind} | VaR: {int(var_level*100)}% ({'hist' if var_method=='historical' else 'param'})")

        print("\nAlpha/Beta y Riesgo de Cola")
        display(capm)

        # Gráfico acumulado
        nav_p = cum_return(port_ret)
        nav_b = cum_return(bench_ret)
        plt.figure(figsize=(10,5))
        plt.plot(nav_p.index, nav_p.values, label="Portafolio")
        plt.plot(nav_b.index, nav_b.values, label=f"Benchmark ({bench_val})", alpha=0.8)
        plt.title("Retorno Acumulado (base=1.0 o Normalizado)")
        plt.ylabel("Crecimiento")
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.show()

# Layout UI
box_left = w.VBox([w.Label("Activos (elige ≥3)"),
                   tickers_w,
                   tickers_extra])

box_mid = w.VBox([w.Label("Benchmark"),
                  bench_dd, bench_txt,
                  start_w, end_w, rf_w,
                  retkind_dd, weights_dd])

box_right = w.VBox([w.Label("Riesgo y Optimización"),
                    var_level_dd, var_method_dd,
                    nstarts_w, seed_w,
                    run_btn])

In [20]:
ui = w.HBox([box_left, box_mid, box_right])
display(ui, out)

run_btn.on_click(run)

Output()

| Métrica          | Valor Óptimo / Bueno                  | Interpretación                                                                 |
|------------------|---------------------------------------|---------------------------------------------------------------------------------|
| **Alpha (α)**    | > 0 (ideal: +2% a +10% anual)         | Exceso de retorno sobre lo esperado por el CAPM; positivo indica valor agregado. |
| **Beta (β)**     | ≈ 1 (mercado), <1 defensivo, >1 agresivo | Sensibilidad al mercado; >1 = más volátil, <1 = más estable.                     |
| **R² (correlación)** | > 0.8 (80% o más) | El portafolio se mueve casi igual que el benchmark (muy “indexado”).|
| **R² (correlación)** | 0.6 – 0.8                          | Buena relación con el mercado, pero con diferencias notables.|
| **R² (correlación)** | < 0.30                         | El portafolio se comporta muy distinto al mercado (alta independencia).|
| **µ (Retorno anual)** | > 8% estable, > 15% agresivo      | Rendimiento esperado anualizado del portafolio.                                 |
| **σ (Volatilidad anual)** | 10%–20% moderado, <10% defensivo, >25% muy riesgoso | Mide el riesgo total (desviación estándar).                                     |
| **Sharpe Ratio** | > 1 bueno, > 1.5 muy bueno, > 2 excelente | Retorno ajustado por riesgo total.                                              |
| **Sortino Ratio**| > 2 excelente                         | Retorno ajustado por riesgo a la baja (mejor si >> Sharpe).                     |
| **VaR (95%, 1d)**| < 2%                                  | Pérdida máxima esperada en un día con 95% de confianza.                         |
| **CVaR (95%, 1d)**| < 3%                                  | Pérdida promedio en los peores días (cola izquierda de la distribución).        |

📊 Pesos del Portafolio
* AAPL (13.8%)
*	GOOG (25.2%)
*	MSFT (61.0%)

👉 El optimizador asignó la mayoría (61%) a Microsoft, seguido por Google y después Apple.
Esto indica que MSFT aporta la mejor combinación de retorno esperado y volatilidad en el periodo analizado, dentro del criterio de maximizar el Sharpe.

⸻

📈 Métricas anualizadas

||	μ anual (rend.)	|σ anual (vol.)|	Sharpe	|Sortino|
|---|---|---|---|-
|Portafolio|	28.8%	|21.9%|	1.32|	2.31|
|Benchmark|	19.5%	|15.5%	|1.26	|NaN|

	•	Retorno anual esperado (μ anual):
	•	Portafolio: 28.8% → supera claramente al benchmark (19.5%).
	•	Volatilidad (σ anual):
	•	Portafolio: 21.9% vs Benchmark: 15.5% → más riesgo, pero controlado.
	•	Sharpe Ratio:
	•	Portafolio: 1.32 vs Benchmark: 1.26 → mejor eficiencia riesgo–retorno.
	•	Un Sharpe > 1 ya es muy bueno; >1.3 se considera excelente.
	•	Sortino Ratio:
	•	Portafolio: 2.31 → mide solo riesgo de caídas.
	•	Al ser mucho más alto que el Sharpe, significa que la mayoría de la volatilidad fue volatilidad “positiva” (subidas).

✅ Conclusión: el portafolio batió al benchmark tanto en retorno como en eficiencia riesgo–ajustada.

⸻

⚖️ Alpha / Beta / Riesgo de cola

||	Alpha (anual)	|Beta	|R²	|VaR 1d (95%)	|CVaR 1d (95%)|
|---|---|---|---|---|---|
|Portafolio|	7.2%|	1.11|	0.613	|2.12%|	2.94%|

*	Alpha (Jensen, 7.2%):
Exceso de rendimiento anualizado respecto a lo que explicaría el CAPM.
👉 Tu portafolio generó rendimiento adicional positivo sobre el benchmark, incluso ajustando por riesgo sistémico.
*	Beta (1.11):
El portafolio es un poco más sensible que el mercado (benchmark).
Si el benchmark sube 1%, tu portafolio tiende a subir 1.11% en promedio.
*	R² (0.613):
~61% de la variación del portafolio se explica por el benchmark.
El 39% restante proviene de factores específicos (stock picking, idiosincráticos).
* VaR 1d (95% = 2.12%):
Con 95% de confianza, la pérdida diaria no debería superar el 2.12% en condiciones normales.
* CVaR 1d (95% = 2.94%):
Si ocurre un evento extremo (peor 5% de escenarios), la pérdida promedio sería 2.94% en un día.

⸻

📝 Interpretación General
* ✅ Eficiencia: Portafolio mejor que el benchmark en Sharpe (1.32 vs 1.26) → más retorno por unidad de riesgo.
* ✅ Exceso de retorno (Alpha): Positivo (7.2%), indica buena selección de activos.
* ⚡ Riesgo: Beta > 1, el portafolio es algo más agresivo que el benchmark; puede amplificar tanto subidas como bajadas.
* 📉 Riesgo de cola: En un mal día (5% de probabilidad), esperas perder ~2–3%.

👉 En resumen: Portafolio bien construido, supera al benchmark en eficiencia y retorno esperado, con un riesgo ligeramente mayor y exposición positiva (alpha). Es atractivo para un inversor dispuesto a aceptar algo más de volatilidad.




## Material Extra
### 📊 Capital Asset Pricing Model (CAPM)

El CAPM (Modelo de Valoración de Activos de Capital) es uno de los modelos más usados en finanzas para estimar el rendimiento esperado de un activo o portafolio en función de su riesgo de mercado.

⸻

🔹 Ecuación principal

El rendimiento esperado de un activo i se calcula como:

$$
E[R_i] = R_f + \beta_i , \big( E[R_m] - R_f \big)
$$

donde:
*	E[R_i] = rendimiento esperado del activo i
*	R_f = tasa libre de riesgo (ej: CETES, bonos del tesoro)
*	E[R_m] = rendimiento esperado del mercado (ej: S&P 500)
*	$\beta_i$ = sensibilidad del activo i frente al mercado (riesgo sistemático)

⸻

🔹 Interpretación
*	$R_f$ → mínimo que deberías ganar sin riesgo.
*	$\beta_i$ → mide cuánto se mueve el activo con el mercado:
*	$\beta=1:$ se mueve igual que el mercado.
*	$\beta>1:$ más volátil (agresivo).
*	$\beta<1:$ menos volátil (defensivo).
*	$\beta<0:$ se mueve al revés del mercado (activo “cobertura”).
*	$E[R_m]-R_f$ → prima de riesgo del mercado.
*	El exceso de retorno del activo depende de su beta.

⸻

🔹 Alpha en CAPM

En la práctica se estima una regresión:

$$
R_i - R_f = \alpha + \beta (R_m - R_f) + \epsilon
$$
* $\alpha$ = rendimiento extra no explicado por el CAPM (habilidad del gestor o ineficiencia de mercado).
*	$\beta$ = exposición al riesgo de mercado.
*	$\epsilon$ = parte aleatoria.

👉 Un alpha positivo significa que el activo o portafolio generó rendimiento adicional más allá de lo que explica su riesgo sistemático.

⸻

🔹 Aplicaciones del CAPM
1.	Valuar activos → estimar retorno esperado en base al riesgo.
2.	Costo de capital → se usa el CAPM para calcular el Costo de Equity (Ke):
$$
K_e = R_f + \beta (E[R_m] - R_f)
$$
3.	Gestión de portafolios → evaluar si un fondo o gestor realmente crea valor (mirando su alpha).
4.	Comparar activos → ayuda a ver si un rendimiento más alto viene de asumir más riesgo o de verdadera eficiencia.

⸻

✅ En resumen:
El CAPM conecta riesgo y retorno: dice que un mayor riesgo sistemático (beta) debe ser recompensado con mayor retorno esperado.
Y el alpha mide si se ganó algo extra fuera de lo que el CAPM predice.
